In [4]:
#Se importan las librerias que se usarán en el proyecto
import pandas as pd 
import numpy as np
from datetime import timedelta

# **Primer parte** 
Aquí se realiza una exploración de los datos, limpieza y homogenización.
Se cruzan las tablas con el fin de tener una sola tabla para responder las preguntas planteadas

In [5]:
#Se importan los csv a dataframes para su transformación y análisis
clientes_df = pd.read_csv("CLIENTES.csv", encoding='utf-8')
transacciones_df = pd.read_csv("TRANSACCIONES.csv", encoding='utf-8')
canales_df = pd.read_csv("CANALES.csv", encoding='utf-8')

# Mostrar las primeras filas para inspección
print(clientes_df.columns)
print(transacciones_df.columns)
print(canales_df.columns)
clientes_df.info()
transacciones_df.info()
canales_df.info()


Index(['tipo_doc', 'num_doc', 'nombre', 'tipo_persona', 'ingresos_mensuales'], dtype='object')
Index(['fecha_transaccion', 'cod_canal', 'tipo_doc', 'num_doc', 'naturaleza',
       'monto'],
      dtype='object')
Index(['codigo', 'nombre', 'tipo', 'cod_jurisdiccion'], dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 701052 entries, 0 to 701051
Data columns (total 5 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   tipo_doc            701052 non-null  object 
 1   num_doc             701052 non-null  int64  
 2   nombre              701052 non-null  object 
 3   tipo_persona        701052 non-null  object 
 4   ingresos_mensuales  701052 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 26.7+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2360614 entries, 0 to 2360613
Data columns (total 6 columns):
 #   Column             Dtype  
---  ------             -----  
 0   fecha_transa

In [6]:
"""Al visualizar los datos, se observa en la tabla de clientes algunos números de documento en negativo,
se transforman dichos datos eliminando el signo negativo, asumiendo que el número se encuentra correcto,
además, se eliminan registros duplicados y clientes con datos incompletos """

clientes_df['num_doc'] = clientes_df['num_doc'].apply(lambda x: np.int64(abs(x)))
clientes_df = clientes_df.drop_duplicates(subset=['num_doc','tipo_doc'])

# Se eliminan los clientes que tienen la información incompleta 
clientes_df.replace('-', np.nan, inplace=True)
clientes_df = clientes_df.dropna()

# Se homogeiniza la columna tipo_persona
clientes_df['tipo_persona'] = clientes_df['tipo_persona'].replace({'NATURAL': 'PERSONA NATURAL'})
clientes_df.info()
clientes_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 698938 entries, 0 to 701051
Data columns (total 5 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   tipo_doc            698938 non-null  object 
 1   num_doc             698938 non-null  int64  
 2   nombre              698938 non-null  object 
 3   tipo_persona        698938 non-null  object 
 4   ingresos_mensuales  698938 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 32.0+ MB


,tipo_doc,num_doc,nombre,tipo_persona,ingresos_mensuales
0,CEDULA DE CIUDADANIA,4421808272321123514,41db2e3894705b1f5297c3c15fcbff159319d718ebc6d3...,PERSONA NATURAL,4784972.0
1,CEDULA DE CIUDADANIA,425016439970959851,b3533bb72a2873d6d17bdea203df0782b2b27a087e7b70...,PERSONA NATURAL,6812197.0
2,TARJETA DE IDENTIDAD,3258301343750323585,8cc728901dd7ed55480c66c1c1f82a09a58a1dcd4722cc...,PERSONA NATURAL,3828842.0
3,CEDULA DE CIUDADANIA,802367903467341407,81dff39fd08eb2f37f1af7f102c1102cf03543a5299190...,PERSONA NATURAL,2890151.0
4,CEDULA DE CIUDADANIA,9053127891032168664,fc71787018999bd2d30be9e7257802592fd948b7e5eaba...,PERSONA NATURAL,788204.0


In [7]:
"""En la tabla de Transacciones el cod_canal se encuentra como decimal, para homogeneizar la información se 
transforma en numero entero. En esta tabla no se eliminan duplicados ya que es una tabla relacional en donde 
un cliente puede realizar muchas transacciones"""

transacciones_df['cod_canal'] = transacciones_df['cod_canal'].astype(int)

# Se transforman los numeros de documento a positivos todos y se transforman los monto de transaccion a positivas
# Ya que se tendrán en cuenta si son ENTRADA O SALIDA
transacciones_df['num_doc'] = transacciones_df['num_doc'].apply(lambda x: np.int64(abs(x)))
transacciones_df['monto'] = transacciones_df['monto'].apply(lambda x: np.int64(abs(x)))

# Se eliminan los clientes que tienen la información incompleta 
transacciones_df.replace('-', np.nan, inplace=True)
transacciones_df = transacciones_df.dropna()

transacciones_df.info()
transacciones_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2314476 entries, 0 to 2360613
Data columns (total 6 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   fecha_transaccion  object
 1   cod_canal          int32 
 2   tipo_doc           object
 3   num_doc            int64 
 4   naturaleza         object
 5   monto              int64 
dtypes: int32(1), int64(2), object(3)
memory usage: 114.8+ MB


,fecha_transaccion,cod_canal,tipo_doc,num_doc,naturaleza,monto
0,2024-05-31,1000527,DOCUMENTO VENEZOLANO/CARNET DIPLOMÁTICO,7274200446135149441,SALIDA,471000
1,2024-05-31,1000527,CEDULA DE CIUDADANIA,7274200446135149441,SALIDA,471000
2,2024-05-31,1000749,NIT,2895732447780163114,SALIDA,3000000
3,2024-05-31,1000545,NIT,2564452540518068244,SALIDA,40900000
4,2024-05-31,1000869,CEDULA DE CIUDADANIA,3684913213504281639,SALIDA,100000


In [8]:
"""Se comprueba que los datos de tipo_doc en las tablas de transacciones 
y clientes estén escritos de la misma manera para que el cruce entre las 
tablas quede correcto"""

print(clientes_df['tipo_doc'].value_counts())
print(transacciones_df['tipo_doc'].value_counts())

CEDULA DE CIUDADANIA                         660104
NIT                                           21871
TARJETA DE IDENTIDAD                           8198
REGISTRO CIVIL                                 5172
CEDULA DE EXTRANJERIA                          2697
PASAPORTE                                       652
DOCUMENTO VENEZOLANO/CARNET DIPLOMÁTICO         237
ID EXTRANJERO PN NO RESIDENTE EN COLOMBIA         5
ID EXTRANJERO PJ NO RESIDENTE COLOMBIA            2
Name: tipo_doc, dtype: int64
CEDULA DE CIUDADANIA                         1949187
NIT                                           326567
TARJETA DE IDENTIDAD                           18713
REGISTRO CIVIL                                 10709
CEDULA DE EXTRANJERIA                           6913
PASAPORTE                                       1794
DOCUMENTO VENEZOLANO/CARNET DIPLOMÁTICO          530
ID EXTRANJERO PN NO RESIDENTE EN COLOMBIA         63
Name: tipo_doc, dtype: int64


In [12]:
"""Unión de tablas para generar una tabla completa con 
toda la información y trabajar a partir de la tabla total"""

# Para evitar confusión con columnas que se llaman igual, se cambia su nombre antes del cruce
canales_df.rename(columns={'nombre': 'nombre_canal'}, inplace=True)
clientes_df.rename(columns={'nombre': 'nombre_persona'}, inplace=True)

# Cruce de las tablas transacciones y canales
tabla_total = pd.merge(
    transacciones_df, 
    canales_df, 
    left_on="cod_canal", 
    right_on="codigo")
#se elimina la columna codigo ya que estaría 2 veces con diferente nombre
tabla_total = tabla_total.drop(['codigo'],axis=1)

# Cruce de la tabla anterior que contiene transacciones y canales con la tabla de clientes
tabla_total = pd.merge(
    tabla_total, 
    clientes_df, 
    left_on =["tipo_doc","num_doc"], 
    right_on=["tipo_doc","num_doc"])
len(tabla_total)

2314456

# **Segunda parte**
Respondiendo a la pregunta: ¿Qué clientes han realizado transacciones en los últimos 6 meses por un monto total superior al 200% 
de sus ingresos mensuales y superiores al percentil 95 del total de la población por tipo de persona?

In [13]:
# Convertir fecha_transaccion a formato datetime
tabla_total['fecha_transaccion'] = pd.to_datetime(tabla_total['fecha_transaccion'])
tabla_total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2314456 entries, 0 to 2314455
Data columns (total 12 columns):
 #   Column              Dtype         
---  ------              -----         
 0   fecha_transaccion   datetime64[ns]
 1   cod_canal           int32         
 2   tipo_doc            object        
 3   num_doc             int64         
 4   naturaleza          object        
 5   monto               int64         
 6   nombre_canal        object        
 7   tipo                object        
 8   cod_jurisdiccion    int64         
 9   nombre_persona      object        
 10  tipo_persona        object        
 11  ingresos_mensuales  float64       
dtypes: datetime64[ns](1), float64(1), int32(1), int64(3), object(6)
memory usage: 220.7+ MB


In [14]:
"""Filtrar transacciones de los últimos 6 meses"""

# Hay 2 tipos de transacciones, entradas y salidas, 
# se filtran solo aquellos de naturaleza salida para realizar el cálculo
transacciones_salida = tabla_total[tabla_total['naturaleza'] == 'SALIDA']

# Se calcula la fecha maxima de transaccion en la tabla 
fecha_maxima = transacciones_salida['fecha_transaccion'].max()

# Se calcula desde esa fecha maxima de transaccion 6 meses hacía atrás
seis_meses_atras = fecha_maxima - timedelta(days=180)

## Se dejan las transacciones que entran dentro del rango de fechas 
trx_ult_6_meses = transacciones_salida[transacciones_salida['fecha_transaccion'] >= seis_meses_atras]
trx_ult_6_meses.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 770659 entries, 0 to 2313490
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   fecha_transaccion   770659 non-null  datetime64[ns]
 1   cod_canal           770659 non-null  int32         
 2   tipo_doc            770659 non-null  object        
 3   num_doc             770659 non-null  int64         
 4   naturaleza          770659 non-null  object        
 5   monto               770659 non-null  int64         
 6   nombre_canal        770659 non-null  object        
 7   tipo                770659 non-null  object        
 8   cod_jurisdiccion    770659 non-null  int64         
 9   nombre_persona      770659 non-null  object        
 10  tipo_persona        770659 non-null  object        
 11  ingresos_mensuales  770659 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int32(1), int64(3), object(6)
memory usage: 73.5+ M

In [17]:
"""Generación de la tabla que contiene los clientes que en los ultimos 6 meses la suma de sus transacciones
exceden almenos el 200% sus ingresos mensuales"""
# Calcular monto total de transacciones por cliente
trx_cliente_exceden = trx_ult_6_meses.groupby(['tipo_doc', 'num_doc']).agg(
    monto_6_meses=pd.NamedAgg(column='monto', aggfunc='sum')).reset_index()


#Union con la tabla de los ultimos 6 meses para traer la información de ingresos 
columnas_trx =trx_ult_6_meses[['tipo_doc', 'num_doc','ingresos_mensuales']]
cliente_exceden = pd.merge(trx_cliente_exceden,columnas_trx, on=['tipo_doc', 'num_doc'], how= 'inner')
cliente_exceden = cliente_exceden.drop_duplicates(subset=['num_doc','tipo_doc'])

#Filtrar clientes que superan el 200% de sus ingresos mensuales
cliente_exceden['excede_200_ingresos'] = cliente_exceden['monto_6_meses'] >= (2 * cliente_exceden['ingresos_mensuales'])
cliente_exceden = cliente_exceden[cliente_exceden['excede_200_ingresos']]
cliente_exceden.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20125 entries, 8 to 770619
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   tipo_doc             20125 non-null  object 
 1   num_doc              20125 non-null  int64  
 2   monto_6_meses        20125 non-null  int64  
 3   ingresos_mensuales   20125 non-null  float64
 4   excede_200_ingresos  20125 non-null  bool   
dtypes: bool(1), float64(1), int64(2), object(1)
memory usage: 805.8+ KB


In [20]:
"""Se genera una lista con los canales usados por cada cliente para las transacciones de salida"""
# Obtener los canales únicos utilizados por cada cliente
canales_usados = trx_ult_6_meses.groupby(['tipo_doc', 'num_doc'])['tipo'].unique().reset_index()
canales_usados['canales'] = canales_usados['tipo'].apply(lambda x: ', '.join(map(str, x)))
canales_usados = canales_usados.drop('tipo', axis=1)
canales_usados.head()

,tipo_doc,num_doc,canales
0,CEDULA DE CIUDADANIA,160655266438380,CAJERO
1,CEDULA DE CIUDADANIA,197031410264756,"CAJERO, SUCURSAL"
2,CEDULA DE CIUDADANIA,463889448627075,"CAJERO, CORRESPONSAL"
3,CEDULA DE CIUDADANIA,1165655920230133,CAJERO
4,CEDULA DE CIUDADANIA,1166368421544241,"SUCURSAL, CAJERO"


In [21]:

# Obtener los canales usados por los clientes en los últimos 6 meses
canales_por_cliente = trx_ult_6_meses.groupby(['tipo_doc', 'num_doc'])['cod_canal'].unique().reset_index()

# Convertir los canales en una lista separada por comas
canales_por_cliente['canales_usados'] = canales_por_cliente['cod_canal'].apply(lambda x: ', '.join(map(str, x)))

# Unir los canales usados con la tabla de clientes que exceden el 200% de sus ingresos
clientes_200= pd.merge(cliente_exceden, canales_usados, on=['tipo_doc', 'num_doc'], how='left')
# Se transforman las columnas a numero entero
clientes_200.head()

,tipo_doc,num_doc,monto_6_meses,ingresos_mensuales,excede_200_ingresos,canales
0,CEDULA DE CIUDADANIA,197031410264756,23745000,2397851.0,True,"CAJERO, SUCURSAL"
1,CEDULA DE CIUDADANIA,463889448627075,14903000,3904492.0,True,"CAJERO, CORRESPONSAL"
2,CEDULA DE CIUDADANIA,2301673292832878,7040000,2903532.0,True,CAJERO
3,CEDULA DE CIUDADANIA,2675004805806660,25740520,7051686.0,True,"SUCURSAL, CAJERO"
4,CEDULA DE CIUDADANIA,2773257023348063,1510000,96154.0,True,"SUCURSAL, CAJERO"


In [23]:
"""Generación de la tabla que contiene los clientes que superan el percentil 95 de transacciones por tipo de persona"""
# Agrupación por cliente y suma del monto de las transacciones para comparar ese monto con el percentil 95.
# Se usa para el calculo del percentil solo las transacciones de salida 
monto_total_por_cliente = transacciones_salida.groupby(['tipo_doc','num_doc','tipo_persona'])['monto'].sum().reset_index()

# Calculo del percentil 95 de transacciones por tipo de persona
percentil_95_por_tipo = monto_total_por_cliente.groupby('tipo_persona')['monto'].quantile(0.95).reset_index()
percentil_95_por_tipo.columns = ['tipo_persona', 'percentil_95']


# Unir el percentil calculado con el Df de monto_total_por_cliente
monto_por_cliente = monto_total_por_cliente.merge(percentil_95_por_tipo, on ='tipo_persona')

# Filtrar los clientes con transacciones superiores al percentil 95
clientes_superan_percentil = monto_por_cliente[monto_por_cliente['monto'] > monto_por_cliente['percentil_95']]
clientes_superan_percentil.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3176 entries, 27 to 63464
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   tipo_doc      3176 non-null   object 
 1   num_doc       3176 non-null   int64  
 2   tipo_persona  3176 non-null   object 
 3   monto         3176 non-null   int64  
 4   percentil_95  3176 non-null   float64
dtypes: float64(1), int64(2), object(2)
memory usage: 148.9+ KB


In [204]:
"""Unión de las tablas de percentiles y monto superior al 200%"""
# Unir percentiles al dataframe de transacciones
trx_porcentil = pd.merge(clientes_200, clientes_superan_percentil, on=['tipo_doc','num_doc'], how='inner')
trx_porcentil.rename(columns={'monto':'monto_total'}, inplace=True)
trx_porcentil.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2881 entries, 0 to 2880
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   tipo_doc             2881 non-null   object 
 1   num_doc              2881 non-null   int64  
 2   monto_6_meses        2881 non-null   int64  
 3   ingresos_mensuales   2881 non-null   float64
 4   excede_200_ingresos  2881 non-null   bool   
 5   canales              2881 non-null   object 
 6   tipo_persona         2881 non-null   object 
 7   monto_total          2881 non-null   int64  
 8   percentil_95         2881 non-null   float64
dtypes: bool(1), float64(2), int64(3), object(3)
memory usage: 205.4+ KB


In [208]:
"""Se guarda la tabla creada en csv para su observación"""
trx_porcentil.to_csv('trx_porcentil.csv', index=False)

In [209]:
# Se exporta con el fin de usarla en las visualizaciones de power bi
tabla_total.to_csv('tabla_total.csv', index=False)